# Data acquisition from .html files

This notebook acquires data from the dataset1

Requirements:

*   dateparser - Enables parsing of dates.

The next cells are needed to run the notebook inside the Google Colaboratory platform, with the datasets in Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# dataset1 location in Google Drive
PATH_falencias = '/content/drive/My Drive/6-Processamento de Linguagem Natural em decisões judiciais/dataset1/falencias'
PATH_recjud = '/content/drive/My Drive/6-Processamento de Linguagem Natural em decisões judiciais/dataset1/recuperacoes_judiciais'
PATH_save = '/content/drive/Shared drives/NLP Jurídico - Lab. Ciência de Dados'


In [ ]:
!pip install dateparser

In [ ]:
#Import packages
import os
import pandas as pd
import csv
import sys
import re
import numpy as np
import matplotlib.pyplot as plt
import dateparser
from dateparser.search import search_dates
#from sklearn.manifold import TSNE
#import matplotlib.pyplot as plt
#import seaborn as sns
import time
from time import sleep
import unidecode
from tqdm import tqdm
import re
import spacy
#Libraries and functions
#import utils
#from utils import *
from dateutil.parser import parse
#Functions
from collections import Counter
import string
#from top2vec import Top2Vec
#import gensim
#import logging
#import scipy.sparse
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#Libraries and functions
#import utils
#from utils import *
#import pyreadr
#import pandas as pd
from bs4 import BeautifulSoup
import lxml
from lxml.html.clean import Cleaner
from lxml.html import fromstring

In [ ]:
#Get data from drive
dataset2f = '/content/drive/Shared drives/NLP Jurídico - Lab. Ciência de Dados/falencias'

root2f, dirs2f, files2f = next(os.walk(dataset2f))
nfiles2f = len(dirs2f)

nfiles2f = 0
size2f = 0
for each_dir in dirs2f:
  sub_path = os.path.join(root2f, each_dir)
  sub_root, sub_dirs, sub_files = next(os.walk(sub_path))
  nfiles2f = nfiles2f + len(sub_files)
  for each_file in sub_files:
    size2f = size2f + os.path.getsize(os.path.join(sub_root, each_file))

"""Coleta de dados dos processos"""


In [ ]:
#Get data from csv uploaded from drive
csv.field_size_limit(sys.maxsize)
dataset2f = '/content/drive/Shared drives/NLP Jurídico - Lab. Ciência de Dados/falencias.csv'

In [ ]:
#Main data
df = pd.read_csv(dataset2f)
df['NumeroDoProcesso']=df['NumeroDoProcesso'].str.replace(']','').str.replace('[','')
df['html']=df['html'].str.replace(']','').str.replace('[','')
print(df.shape)
df.head()

In [ ]:
#Cleaning
for i in range(0,len(df)):
    
    soup = BeautifulSoup(df['html'][i])

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    text = "".join(text.split('\\n'))
    text = "".join(text.split('\\t')).replace('\\n',' ')
    #text = "".join(text.split('\n'))
    text = text.replace('\\xa0','')
    text = text.replace('Advogada','Advogado')
    df['html'][i] = text.lower()


In [ ]:
#New dataframe
dfinal = pd.DataFrame(columns=['num_proc','status','juiz','terceiros','rep_legal','valor_acao','reqte','adv_reqte','reqdo','adv_reqdo'])
errors=[]
for i in range(0,len(df)):
  #print(i)
  try:
  #Numero processo e satus
    a="".join(re.findall("[\d.-]", df['html'][i].replace('\n','').split('dados do processo')[1].split('processo:')[1].split('classe:')[0]))
    #s="".join(re.findall("[^\d\W]", df['html'][i].replace('\n','').split('dados do processo')[1].split('processo:')[1].split('classe:')[0]))
    s="".join(re.findall("[^\d:.,-]+", df['html'][i].replace('\n','').split('dados do processo')[1].split('processo:')[1].split('classe:')[0]))
  except:
    a=''
    s=''
    errors+=[i]

  try:

    t=df['html'][i].split('partes do processo')[1].split('movimentações')[0].split('terintcer:')[1].split('\n')[0]
    r=",".join(list(filter(lambda x: 'repreleg:' in x,df['html'][i].split('reqdo')[1].split('\n'))))
    r=r.replace('repreleg:','')
  except:
    t=r=''

  try:
    #Juiz, terceiros, representante
    #b=df['html'][i].split('dados do processo')[1].split('juiz:')[1].split('valor da ação:')[0].replace('\n','')
    if 'outros' in df['html'][i].split('dados do processo')[1].split('juiz:')[1].split('valor da ação:')[0]:
      b=" ".join(re.findall(r'[^\s0-9-,.]+',df['html'][i].split('dados do processo')[1].split('juiz:')[1].split('valor da ação:')[0])[:-2])
    else:
      b=" ".join(re.findall(r'[^\s0-9-,.]+',df['html'][i].split('dados do processo')[1].split('juiz:')[1].split('valor da ação:')[0]))

  except:
    b=''
    errors+=[i]
  #Valor da ação
  try:
    c=re.findall("r[$]\d{1,3}.\d{3}\,\d{2}", df['html'][i].replace('\n','').split('dados do processo')[1].split('processo:')[1].split('valor da ação:')[1])[0]
    c=c.replace('r$','')
    #c=re.findall("[r$]\d{1,3}.\d{3}\,\d{2}", df['html'][i].replace('\n','').split('dados do processo')[1].split('processo:')[1].split('valor da ação:')[1])[0]
  except:
    c=''
    errors+=[i]
  try:
    if 'reqdo' in df['html'][i]:
      x='reqdo'
    else:
      x='falido'
    #Requerente
    d=" ".join(re.findall(r'[^\s0-9-,.]+',df['html'][i].split('reqte:')[1].split('advogado')[0]))
    #d=df['html'][i].split('reqte:')[1].split('advogado')[0]
    #Advogado Requerente
    e=" ".join(re.findall(r'[^\s0-9-,.:]+',df['html'][i].split('reqte:')[1].split('advogado')[1].split(x)[0]))
    #e=df['html'][i].split('reqte:')[1].split('advogado')[1].split(x)[0]
    #Requerido
    #f=" ".join(re.findall(r'[^\s0-9-,.:]+',df['html'][i].split(x)[1].split('advogado')[0]))
    #f=df['html'][i].split(x)[1].split('advogado')[0]
    f=re.findall(r'[^0-9-,.:]+',df['html'][i].split(x)[1].split('\n')[0])[0]
    #Advogado Requerido
    g=",".join(list(filter(lambda x: 'advogado:' in x,df['html'][i].split(x)[1].split('\n'))))
    g=g.replace('advogado:','')
   
  except:
    errors+=[i]
    d=e=f=g=''

  dfinal=dfinal.append({'num_proc':a,'status':s,'juiz':b,'terceiros':t,'rep_legal':r,'valor_acao':c,'reqte':d,'adv_reqte':e,'reqdo':f,'adv_reqdo':g},ignore_index=True)
  errors=list(set(errors))


In [ ]:
dfinal